# Equity Risk Factor Modelling

## Setup

Assuming some input parameters

In [ ]:
import numpy as np 
import polars as pl 
import matplotlib.pyplot as plt 


# Model parameters
S0: float = 100.0 # initial spot 
r: float = 0.02   # risk-free rate 
q: float = 0.01   # dividend yield 
T: float = 1.0    # horizon (years) 
n_paths: int = 10_000 # number of Monte Carlo paths 
n_steps: int = 252    # Number of steps in one year
time_grid = np.linspace(start=0.0, stop=T, num=n_steps + 1) # Number of interval is n_steps but number of points is n_steps + 1 

# Setting Seed
rng= np.random.default_rng(seed=42) 

# Polars DataFrame to hold simulated results 
results_schema: dict[str,any] = { "path_id": pl.Int32, "time": pl.Float64, "S": pl.Float64 } 
simulated_paths = pl.DataFrame(schema=results_schema) 

# Plot style
plt.style.use("seaborn-v0_8-whitegrid") 
print("Setup complete. Polars and parameters initialized.")

Setup complete. Polars and parameters initialized.


# Basic Model 
This will be used by Heston, SABR or Local Volatility

In [3]:
from abc import ABC, abstractmethod
import polars as pl
import numpy as np

class BaseModel(ABC):
    """Abstract base class for equity models."""

    def __init__(self, S0: float, r: float, q: float, T: float, n_paths: int, n_steps: int, rng: np.random.Generator) -> None:
        self.S0 = S0
        self.r = r
        self.q = q
        self.T = T
        self.n_paths = n_paths
        self.n_steps = n_steps
        self.dt = T / n_steps
        self.rng = rng

    @abstractmethod
    def simulate_paths(self) -> pl.DataFrame:
        """Simulate asset price paths and return as Polars DataFrame."""
        pass

# Monte Carlo Engine
Defines the engine

In [ ]:
import numpy as np
import polars as pl

class MonteCarloEngine:
    """Monte Carlo engine for simulating Brownian paths."""

    def __init__(self, T: float, n_paths: int, n_steps: int, rng: np.random.Generator) -> None:
        self.T = T
        self.n_paths = n_paths
        self.n_steps = n_steps
        self.dt = T / n_steps
        self.sqrt_dt = np.sqrt(self.dt)
        self.rng = rng
        self.time_grid = np.linspace(0.0, T, n_steps + 1)

    def generate_brownian_increments(self) -> np.ndarray:
        """
        Generate standard Brownian motion increments dW for all paths and steps.
        Each path i has a sequence of random increments over time steps j:

            dW =
                   step_1   step_2   ...  step_N
            path_1   x11      x12    ...   x1N
            path_2   x21      x22    ...   x2N
            ...      ...      ...    ...   ...
            path_M   xM1      xM2    ...   xMN

        Each element x_ij represents one Brownian increment:
            dW_t = N(0,1) * sqrt(dt)

        Returns: array of shape (n_paths, n_steps)
        """
        dW = self.rng.standard_normal((self.n_paths, self.n_steps)) * self.sqrt_dt
        return dW

    def generate_brownian_paths(self) -> np.ndarray:
            """
            Generate Brownian motion paths W_t by cumulatively summing the increments dW.

            Each path i accumulates its increments over time steps j:

                dW =
                       step_1   step_2   ...  step_N
                path_1   x11      x12    ...   x1N
                path_2   x21      x22    ...   x2N
                ...      ...      ...    ...   ...
                path_M   xM1      xM2    ...   xMN

            The Brownian motion W_t is built as the cumulative sum of these increments.
            Including W_0 = 0

                W =
                        t0     t1       t2      ...    tN
                path_1   0    x11     x11+x12   ...   Sum x1j
                path_2   0    x21     x21+x22   ...   Sum x2j
                ...      ...    ...      ...    ...    ...
                path_M   0    xM1     xM1+xM2   ...   Sum xMj

            Mathematically:
                W_t = Sum_{k=1}^{t/dt} (N(0,1) * sqrt(dt))

            Returns: array of shape (n_paths, n_steps + 1)
            """
            dW = self.generate_brownian_increments()
            W = np.zeros((self.n_paths, self.n_steps + 1))
            W[:, 1:] = np.cumsum(dW, axis=1)
            return W



# Local Volatility Model

Simulation

Risk Metrics